In [1]:
from sklearn import datasets
import pandas as pd
import numpy as np
iris = datasets.load_iris()
import random
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))
from src.models_fn.imputer_models import measure_val_error,XGBImputer,MySimpleImputer
from src.data_fn.data_process import simulate_missing_values,replace_infs,test_input_data,infer_cols
from pyTest.test_suite import get_test_data
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.metrics import mean_squared_error,f1_score
import warnings
warnings.filterwarnings("ignore")
from sklearn.impute import SimpleImputer

Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\models_fn\imputer_models.py
Importing C:\Users\johma\Desktop\main\4fun\AutoImputer\src\data_fn\data_process.py


In [2]:
data = pd.read_csv('..\data\processed\iris_original.csv')
curr_df = simulate_missing_values(data,output_name=None,prop=0.2)
curr_df = pd.read_csv('..\data\processed\iris_nans.csv')
inf_df = curr_df.copy()
# inf_df['target'] = inf_df['target'].map({0.0:'A',1.0:'B',2.0:'C'})
# inf_df.iloc[0,2] = -np.inf
dtypes = ["numeric", "numeric", "numeric", "numeric", "categorical"]
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
imp1 = MySimpleImputer(dtype_list=dtype_list)


In [3]:
data = pd.read_csv(r'..\data\processed\adult_nans_02_2k.csv')
# data.loc[1:2000,:].to_csv(r'..\data\processed\adult_nans_02.csv',index=False)
dtypes = ["numeric", "categorical", "numeric", "categorical", "numeric", "categorical", "categorical", "categorical", "categorical", "categorical",'numeric','numeric','numeric','categorical','categorical']
cols = data.columns
dtype_list = dict(zip(cols, dtypes))
data

,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,NaN,White,Male,0.0,NaN,40.0,United-States,<=50K
1,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,NaN
2,NaN,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,NaN,NaN,40.0,Cuba,<=50K
3,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,NaN,NaN,NaN,NaN,NaN,0.0,40.0,United-States,<=50K
4,49.0,Private,NaN,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,NaN,0.0,16.0,Jamaica,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,NaN,Self-emp-not-inc,30599.0,Masters,14.0,Married-spouse-absent,Prof-specialty,NaN,White,Female,0.0,0.0,50.0,United-States,<=50K
1996,37.0,Private,379522.0,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,55.0,United-States,>50K
1997,51.0,NaN,NaN,HS-grad,9.0,NaN,Craft-repair,Unmarried,NaN,Male,0.0,0.0,38.0,United-States,NaN
1998,35.0,Private,82552.0,NaN,NaN,Never-married,NaN,Unmarried,White,Male,0.0,0.0,35.0,United-States,NaN


In [4]:
# # imp1 = MySimpleImputer(dtype_list=dtype_list)
# imp1 = XGBImputer(dtype_list=dtype_list)
imp1 = MySimpleImputer(dtype_list=dtype_list)
res = imp1.impute(data)
res


,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
0,38.0,Private,215646.0,HS-grad,9.0,Divorced,Handlers-cleaners,Husband,White,Male,0.0,91.582543,40.0,United-States,<=50K
1,53.0,Private,234721.0,11th,7.0,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0.0,0.0,40.0,United-States,<=50K
2,38.855466,Private,338409.0,Bachelors,13.0,Married-civ-spouse,Prof-specialty,Wife,Black,Female,999.982901,91.582543,40.0,Cuba,<=50K
3,37.0,Private,284582.0,Masters,14.0,Married-civ-spouse,Craft-repair,Husband,White,Male,999.982901,0.0,40.0,United-States,<=50K
4,49.0,Private,193085.347442,9th,5.0,Married-spouse-absent,Other-service,Not-in-family,Black,Female,999.982901,0.0,16.0,Jamaica,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,38.855466,Self-emp-not-inc,30599.0,Masters,14.0,Married-spouse-absent,Prof-specialty,Husband,White,Female,0.0,0.0,50.0,United-States,<=50K
1996,37.0,Private,379522.0,Bachelors,13.0,Married-civ-spouse,Sales,Husband,White,Male,0.0,0.0,55.0,United-States,>50K
1997,51.0,Private,193085.347442,HS-grad,9.0,Married-civ-spouse,Craft-repair,Unmarried,White,Male,0.0,0.0,38.0,United-States,<=50K
1998,35.0,Private,82552.0,HS-grad,10.150531,Never-married,Craft-repair,Unmarried,White,Male,0.0,0.0,35.0,United-States,<=50K


In [5]:
measure_val_error(data,imp1)

{'39': 9.51,
 ' State-gov': 0.86,
 ' 77516': 54229.39,
 ' Bachelors': 0.74,
 ' 13': 1.29,
 ' Never-married': 0.83,
 ' Adm-clerical': 0.7,
 ' Not-in-family': 0.73,
 ' White': 0.95,
 ' Male': 0.88,
 ' 2174': 911.08,
 ' 0': 364.38,
 ' 40': 9.22,
 ' United-States': 0.95,
 ' <=50K': 0.93}

In [6]:
low_cardinality_cols = [
    col
    for col in data.columns
    if (data[col].nunique() / data[col].count() < 0.05) and (1.*data[col].value_counts(normalize=True).head(3).sum() > 0.9)
]
low_cardinality_cols

[' Never-married',
 ' White',
 ' Male',
 ' 2174',
 ' 0',
 ' United-States',
 ' <=50K']

In [20]:
infer_cols(inf_df)

([], {'target'})

In [23]:
test_df = inf_df
text_cols = [
    x
    for x in test_df.columns
    if (test_df[x].dtype == object) and (isinstance(test_df.dropna().iloc[0][x], str))
]
low_cardinality_cols = [
    col
    for col in test_df.columns
    if (test_df[col].nunique() / test_df[col].count() < 0.05)
]
infer_cat_cols = set(text_cols + low_cardinality_cols)

In [24]:
low_cardinality_cols

['target']

In [25]:
# (test_df[' 0'].nunique() / test_df[' 0'].count() < 0.08)
# test_df[]
test_df['target'].nunique() / test_df['target'].count() # 0.02702702702702703
test_df['target'].nunique() / test_df['target'].count()
test_df['target'].nunique() / test_df['target'].count()


0.02702702702702703

In [18]:
test_df[' 13'].value_counts()

9.0     538
10.0    334
13.0    270
14.0     98
11.0     71
7.0      64
12.0     55
6.0      49
4.0      27
15.0     24
5.0      20
16.0     18
3.0      12
8.0      12
2.0       7
1.0       2
Name:  13, dtype: int64

In [12]:
data[' 2174'].value_counts()

0.0        1434
15024.0      19
7688.0       15
7298.0       14
3103.0        8
5178.0        7
99999.0       6
1055.0        6
4386.0        6
3325.0        4
2174.0        4
594.0         4
3411.0        4
2407.0        3
5013.0        3
8614.0        3
14084.0       3
10605.0       2
4650.0        2
2105.0        2
2176.0        2
2202.0        2
1409.0        2
4064.0        2
25124.0       1
10520.0       1
3471.0        1
6514.0        1
6849.0        1
3674.0        1
1471.0        1
1506.0        1
4865.0        1
2597.0        1
2580.0        1
2964.0        1
4101.0        1
9386.0        1
25236.0       1
1111.0        1
20051.0       1
2050.0        1
3464.0        1
34095.0       1
14344.0       1
13550.0       1
Name:  2174, dtype: int64

In [13]:
infer_cols(data)

([' State-gov',
  ' Bachelors',
  ' Never-married',
  ' Adm-clerical',
  ' Not-in-family',
  ' White',
  ' Male',
  ' United-States',
  ' <=50K'],
 {' 13',
  ' <=50K',
  ' Adm-clerical',
  ' Bachelors',
  ' Male',
  ' Never-married',
  ' Not-in-family',
  ' State-gov',
  ' United-States',
  ' White'})

In [14]:
data[' 0'].value_counts()
data[' 0'].nunique()

35